3d rotation is way to increase numer of 2d slices for training.

**WARNING:** original 3d image should have approximately equal axes scales or new slices would be highly distorted.

In [ ]:
import os, re, pydicom
import numpy as np
from scipy.ndimage import affine_transform
import matplotlib.pyplot as plt

In [ ]:
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

In [ ]:
def plot3d(img3d):
    fig, ax = plt.subplots(1,3)
    fig.set_figheight(10)
    fig.set_figwidth(20)
    
    ax[0].imshow(img3d[:,:,img3d.shape[2]//2])
    ax[1].imshow(img3d[:,img3d.shape[1]//2,:])
    ax[2].imshow(img3d[img3d.shape[0]//2,:,:])
    plt.show()

def img3d_from_dicom(path):
    files = sorted(os.listdir(path), key=lambda x: int(re.findall('\\d+', x)[0]))
    deepth = len(os.listdir(path))
    
    for idx, f in enumerate(files):
        dicom = pydicom.read_file(path + f)
        data = dicom.pixel_array.astype(np.float32)
           
        if idx > 0:
            img3d[:,:,idx] = data
        else:            
            img3d = np.zeros((data.shape[0], data.shape[1], deepth), dtype=np.float32)
            img3d[:,:,0] = data
        
    return img3d - np.min(img3d)

In [ ]:
# Rotation vector (rot_v) is a three-dimensional vector 
# whose direction is the axis of rotation and 
# whose magnitude is the angle in radians.

def rotate_by_vector(img3d, rot_v):
    
    x, y, z = rot_v
    angle = np.sqrt(x*x + y*y + z*z)
    cos = np.cos(angle)
    sin = np.sin(angle)
    x, y, z = rot_v / angle

    R = np.eye(4, dtype=np.float32)
    R[0, 0] = cos + (1 - cos)*x*x
    R[1, 1] = cos + (1 - cos)*y*y
    R[2, 2] = cos + (1 - cos)*z*z
    R[0, 1] = (1 - cos)*x*y - sin*z
    R[0, 2] = (1 - cos)*x*z + sin*y    
    R[1, 0] = (1 - cos)*x*y + sin*z
    R[1, 2] = (1 - cos)*y*z - sin*x
    R[2, 0] = (1 - cos)*x*z - sin*y
    R[2, 1] = (1 - cos)*y*z + sin*x
    
    T1 = np.array(img3d.shape) / 2
    T2 = np.matmul(R[:3,:3], T1)
    R[:3, 3] = T1 - T2
   
    return affine_transform(img3d, R)

In [ ]:
patient = '00000'
path = data_directory + '/train/' + patient + '/FLAIR/'

img3d  = img3d_from_dicom(path)[80:400, 110:410, 40:350]
plot3d(img3d)

In [ ]:
img3d_r = rotate_by_vector(img3d, np.radians([0, 10, 0]))
plot3d(img3d_r)

In [ ]:
img3d_r = rotate_by_vector(img3d, np.radians([-5, 10, -2]))
plot3d(img3d_r)